# Examples of usage of callbacks in Keras

- Save trained model and load it to continue training.


In [ ]:
#First define a model
from keras.datasets import mnist
from keras.layers import Input, Dense
from keras.models import Model
from keras.utils import np_utils

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

batch_size = 64
nb_classes = 10
nb_epoch = 4


# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)



# Model
inputs = Input(shape=(784,))

x = Dense(64, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

model = Model(input=inputs, output=predictions)

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# train the model
model.fit(X_train, Y_train,
          batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_test, Y_test))



In [ ]:
# Model checkpoint
from keras.callbacks import ModelCheckpoint
import pickle

model_path = '/tmp/keras/'

#Save model structure
print 'Saving architecture...'
json_string = model.to_json()
open(model_path + 'model_architecture.json', 'w').write(json_string)

#TRain and save checkpoints
print 'Train...'
checkpointer = ModelCheckpoint(filepath=model_path+"weights.{epoch:02d}-{val_acc:.5f}.hdf5", verbose=1)
history = model.fit(X_train, Y_train,
          batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_test, Y_test),
          callbacks=[checkpointer])

#Save history
pickle.dump( history.history, open( model_path + "history.p", "wb" ) )

print 'Done!'

In [ ]:
for h in history.history.iteritems():
    print h

In [ ]:
# Train starting on a previous weigths
from keras.models import model_from_json
from keras.optimizers import SGD

model3 = model_from_json(open(model_path + 'model_architecture.json').read())

model3.load_weights(model_path + 'weights.03-0.97610.hdf5')

model3.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    
history2 = model3.fit(X_train, Y_train,
                      batch_size=batch_size, nb_epoch=nb_epoch,
                      validation_data=(X_test, Y_test))